In [1]:
import numpy as np
import regex as re
import utils
import textwrap
import parsimonious

In [2]:
with open('STAMPS MPOT v32g.dyn','rU') as in_file:
    text=in_file.read()
text=re.sub(r'(?i)MACRO[\s\S]*MEND','',text)
text=re.sub(r'\*+ {2}Group.*','',text)

In [3]:
#deprecated, too slow
def if_then_else(text):
    try:
        valuetounpack=re.findall('(?i)(?!.{2}.*?if)(if.*?then.*?else.*?)(then|else|$)',text)[-1][0]
        unpacked=filter(None,[x.strip() for x in re.split(r'(?i)if|then|else',valuetounpack)])
        text=text.replace(valuetounpack,'_conditional(%s)'%','.join(unpacked))
        return if_then_else(text)
    except:
        return text
    
def if_then_else(text):
    try:
        valuetounpack=re.findall('(?i)((\W|if|then|else) *)(if.*?then.*?else.*?)(then|else|$)',
                                 text,overlapped=True)[-1][2]
        unpacked=filter(None,[x.strip() for x in re.split(r'(?i)then|else',valuetounpack[2:])])
        text=text.replace(valuetounpack,'_conditional(%s)'%','.join(unpacked))
        return if_then_else(text)
    except:
        return text    

In [24]:
functiondic={'MAX':'np.maximum','MIN':'np.minimum','SUM':'np.sum','if_then_else':'np.where',
             'SUMV':'functions.sumv','ABS':'abs', 'INTEGER':'int','EXP':'np.exp','PI':'np.pi', 
             'SIN':'np.sin', 'COS':'np.cos','SQRT':'np.sqrt', 'TAN':'np.tan', 'LOGN':'np.random.lognormal',
             'RANDOM NORMAL':'functions.bounded_normal','POISSON':'np.random.poisson','LN':'np.log', 
             'EXPRND':'np.random.exponential','RANDOM UNIFORM':'np.random.rand','ARCCOS':'np.arccos',
             'ARCSIN':'np.arcsin','ARCTAN':'np.arctan','STEP':'functions.step', 'MODULO':'np.mod', 
             'PULSE':'functions.pulse', 'PULSE TRAIN':'functions.pulse_train','RAMP':'functions.ramp', 
             'FIFZE':'functions.fifze', 'FIFGE':'functions.fifge','SCLPRD':'functions.sclprd', 
             'PRDV':'functions.prdv','SMOOTH3I':'smooth3i','SMOOTH3':'smooth3','SMOOTHI':'smoothi',
             'SMOOTH':'smooth','LOGN':'np.log', 'ROUND':'np.round','DELAY3I':'delay3i','DELAY3':'delay3',
             'DELAY1I':'delayi','DELAY1':'delay','DELAYI':'delayi','DELAY':'delay','FLOOR':'np.floor',
             'VECTORMIN':'np.min','VECTORMAX':'np.max','PROD':'np.prod','TABHL':'functions.tabhl',
             'POS':'pos','MISSING()':'0','TUNE1':'tune1','_conditional':'if_then_else'
            }
functiondic={x.lower():functiondic[x].lower() for x in functiondic.keys()}

In [49]:
model_all=[x[0] for x in re.findall(r'([A-Z]+ +.+(\n {5}.+)*)',text)]
model_vars=[x for x in model_all if re.match('(C|A|T|R|L|N|SPEC)\s',x)]

#Subscripts
model_subs=[re.sub(r'\n\s{5}','',x[5:]) for x in model_all if x.startswith('FOR')]
model_subs={x.split('=',1)[0]:x.split('=',1)[-1].split(',') for x in model_subs}
for key,value in model_subs.iteritems():
    if len(value)==1 and re.search('-',value[0]):
        value=value[0]
        if not re.search(r'[a-zA-Z]',value):
            start=int(value.split('-')[0])
            stop=int(value.split('-')[-1])+1
            model_subs[key]=[str(x) for x in range(start,stop)]
        else:
            start=value.split('-')[0]
            stop=value.split('-')[-1]
            base=[x for x,y in model_subs.iteritems() if x!=key and start in y and stop in y]
            base=model_subs[base[0]]
            model_subs[key]=[x for x in base if base.index(x)>=base.index(start) 
                             and base.index(x)<=base.index(stop)]

model_subs_list=[str(x) for x in set(reduce(lambda x,y:x+y,model_subs.values())+model_subs.keys()+[':'])]
model_subs_list='|'.join(model_subs_list)

In [50]:
model_list=[str(x) for x in set(reduce(lambda x,y:x+y,model_subs.values())+model_subs.keys())]
[model_list.append(re.findall(r'[A-Z]+ +([^=\(]+)[=\(]',x)[0]) for x in model_vars]
[model_list.append(x) for x in functiondic.keys()]
model_list=[x.lower() for x in model_list]
model_list=set(model_list)

In [51]:
def cleanup(function,compareset):
    number=r'[e\d]+'
    func=''.join(function)
    for variable in re.split(r'[^\w ]+',func.split(' ',1)[-1].strip()):
        variable=variable.strip().lower()
        if len(function)>1 and variable:
            if re.match(r'\(\W+\)',function[-1]) \
            or variable not in compareset \
            and not re.match(number,variable) \
            and not re.search('if|then|else',variable):
                return cleanup(function[:-1],compareset)
    return func

In [52]:
# %%prun
classify={'C':'constants','A':'auxiliaries','T':'tables','R':'rates',
          'L':'levels','N':'initials','SPEC':'specs'}

model=[x.strip().split('\n     ') for x in model_vars] #OR MAYBE {6}
model=[cleanup(x,model_list) for x in model]
model=[re.sub(r'(\()([\w ]+,)*(\*)(,[\w ]+)*(\))',r'\1\2:\4\5',x) for x in model]
model=[re.sub(r'(?i)\(((%s)(,(%s))*)\)'%(model_subs_list,model_subs_list),r'[\1]',x) for x in model]
model=[if_then_else(x) for x in model]
model=[re.sub(r'([\w ]+\=)([\w ]+\+ *DT *\*)(\(.*\))',r'\1INTEG\3',x) for x in model]
datadictionary={classify[x]:[y.split(' ',1)[-1].strip() for y in model 
                             if re.match('\w+',y).group()==x] 
                for x in set(classify.keys())}
# model=[x.split(' ',1)[-1].strip() for x in model]

In [42]:
x='sum(Retention Costs(*)) '
re.sub(r'(\()([\w ]+,)*(\*)(,[\w ]+)*(\))',r'\1\2:\4\5',x)

'sum(Retention Costs(:)) '

In [53]:
components=['kind','real_name','subs','expr']
entries=[dict(zip(components,list(re.match(r'(\w+) +([\w ]+)\[?(.*?)\]?\=(.*)',x).groups()))) for x in model]
for x in entries:
    x.update({'subs':filter(None,x['subs'].split(','))})

In [54]:
namespace = {'TIME': 'time', 'Time': 'time'}  # Initialize with builtins
for element in entries:
    if element['kind'] not in ['T', 'FOR']:
        element['py_name'], namespace = utils.make_python_identifier(element['real_name'],
                                                                     namespace)

In [57]:
parse_general_expression(entries[19],namespace,model_subs)
# entries[19]

ParseError: Rule 'sub_element' didn't match at ':]) ' (line 1, column 21).

In [14]:
def parse_general_expression(element, namespace=None, subscript_dict=None):
    """
    Parses a normal expression
    # its annoying that we have to construct and compile the grammar every time...

    Parameters
    ----------
    element: dictionary

    namespace : dictionary

    subscript_dict : dictionary


    Returns
    -------
    translation

    new_elements: list of dictionaries
        If the expression contains builder functions, those builders will create new elements
        to add to our running list (that will eventually be output to a file) such as stock
        initialization and derivative funcs, etc.


    Examples
    --------
    >>> parse_general_expression({'expr': 'INTEG (FlowA, -10)',
    ...                           'py_name':'test_stock',
    ...                           'subs':None},
    ...                          {'FlowA': 'flowa'}),
    ({'kind': 'component', 'py_expr': "_state['test_stock']"},
     [{'kind': 'implicit',
       'subs': None,
       'doc': 'Provides initial conditions for test_stock function',
       'py_name': 'init_test_stock',
       'real_name': None,
       'unit': 'See docs for test_stock',
       'py_expr': '-10'},
      {'py_name': 'dtest_stock_dt',
       'kind': 'implicit',
       'py_expr': 'flowa',
       'real_name': None}])

    """
    if namespace is None:
        namespace = {}
    if subscript_dict is None:
        subscript_dict = {}

    functions = functiondic

    builders = {
        "integ": lambda expr, init: builder.add_stock(element['py_name'], element['subs'],
                                                      expr, init, subscript_dict),
        "delay1": lambda in_var, dtime: builder.add_n_delay(in_var, dtime, '0', '1',
                                                            element['subs'], subscript_dict),
        "delay1i": lambda in_var, dtime, init: builder.add_n_delay(in_var, dtime, init, '1',
                                                                   element['subs'], subscript_dict),
        "delay3": lambda in_var, dtime: builder.add_n_delay(in_var, dtime, '0', '3',
                                                            element['subs'], subscript_dict),
        "delay3i": lambda in_var, dtime, init: builder.add_n_delay(in_var, dtime, init, '3',
                                                                   element['subs'], subscript_dict),
        "delay n": lambda in_var, dtime, init, order: builder.add_n_delay(in_var, dtime,
                                                                          init, order,
                                                                          element['subs'],
                                                                          subscript_dict),
        "smooth": lambda in_var, dtime: builder.add_n_smooth(in_var, dtime, '0', '1',
                                                             element['subs'], subscript_dict),
        "smoothi": lambda in_var, dtime, init: builder.add_n_smooth(in_var, dtime, init, '1',
                                                                    element['subs'],
                                                                    subscript_dict),
        "smooth3": lambda in_var, dtime: builder.add_n_smooth(in_var, dtime, '0', '3',
                                                              element['subs'], subscript_dict),
        "smooth3i": lambda in_var, dtime, init: builder.add_n_smooth(in_var, dtime, init, '3',
                                                                     element['subs'],
                                                                     subscript_dict),
        "smooth n": lambda in_var, dtime, init, order: builder.add_n_smooth(in_var, dtime,
                                                                            init, order,
                                                                            element['subs'],
                                                                            subscript_dict),
        "initial": lambda initial_input: builder.add_initial(initial_input)
    }

    in_ops = {
        "+": "+", "-": "-", "*": "*", "/": "/", "^": "**", "=": "==", "<=": "<=", "<>": "!=",
        "<": "<", ">=": ">=", ">": ">",
        ":and:": " and ", ":or:": " or "}  # spaces important for word-based operators

    pre_ops = {
        "-": "-", ":not:": " not ",  # spaces important for word-based operators
        "+": " "  # space is important, so that and empty string doesn't slip through generic
    }

    # in the following, if lists are empty use non-printable character
    # everything needs to be escaped before going into the grammar, in case it includes quotes
    sub_names_list = [re.escape(x) for x in subscript_dict.keys()] or ['\\a']
    sub_elems_list = [re.escape(y) for x in subscript_dict.values() for y in x] or ['\\a']
    ids_list = [re.escape(x) for x in namespace.keys()] or ['\\a']
    in_ops_list = [re.escape(x) for x in in_ops.keys()]
    pre_ops_list = [re.escape(x) for x in pre_ops.keys()]

    expression_grammar = r"""
    expr_type = array / expr
    expr = _ pre_oper? _ (lookup_def / build_call / call / parens / number / reference) _ (in_oper _ expr)?

    lookup_def = ~r"(WITH\ LOOKUP)"I _ "(" _ reference _ "," _ "(" _  ("[" ~r"[^\]]*" "]" _ ",")?  ( "(" _ expr _ "," _ expr _ ")" ","? _ )+ _ ")" _ ")"
    call = (func / id) _ "(" _ (expr _ ","? _)* ")" # allows calls with no arguments
    build_call = builder _ "(" _ (expr _ ","? _)* ")" # allows calls with no arguments
    parens   = "(" _ expr _ ")"

    reference = id _ subscript_list?
    subscript_list = "[" _ ((sub_name / sub_element) _ ","? _)+ "]"

    array = (number _ ("," / ";")? _)+ !~r"."  # negative lookahead for anything other than an array
    number = ~r"\d+\.?\d*(e[+-]\d+)?"

    id = ~r"(%(ids)s)"I
    sub_name = ~r"(%(sub_names)s)"I  # subscript names (if none, use non-printable character)
    sub_element = ~r"(%(sub_elems)s)"I  # subscript elements (if none, use non-printable character)

    func = ~r"(%(funcs)s)"I  # functions (case insensitive)
    in_oper = ~r"(%(in_ops)s)"I  # infix operators (case insensitive)
    pre_oper = ~r"(%(pre_ops)s)"I  # prefix operators (case insensitive)
    builder = ~r"(%(builders)s)"I  # builder functions (case insensitive)

    _ = ~r"[\s\\]*"  # whitespace character
    """ % {
        # In the following, we have to sort keywords in decreasing order of length so that the
        # peg parser doesn't quit early when finding a partial keyword
        'sub_names': '|'.join(reversed(sorted(sub_names_list, key=len))),
        'sub_elems': '|'.join(reversed(sorted(sub_elems_list, key=len))),
        'ids': '|'.join(reversed(sorted(ids_list, key=len))),
        'funcs': '|'.join(reversed(sorted(functions.keys(), key=len))),
        'in_ops': '|'.join(reversed(sorted(in_ops_list, key=len))),
        'pre_ops': '|'.join(reversed(sorted(pre_ops_list, key=len))),
        'builders': '|'.join(reversed(sorted(builders.keys(), key=len))),
    }

    parser = parsimonious.Grammar(expression_grammar)
    tree = parser.parse(element['expr'])

    class ExpressionParser(parsimonious.NodeVisitor):
        # Todo: at some point, we could make the 'kind' identification recursive on expression,
        # so that if an expression is passed into a builder function, the information
        # about whether it is a constant, or calls another function, goes with it.
        def __init__(self, ast):
            self.translation = ""
            self.kind = 'constant'  # change if we reference anything else
            self.new_structure = []
            self.visit(ast)

        def visit_expr_type(self, n, vc):
            s = ''.join(filter(None, vc)).strip()
            self.translation = s

        def visit_expr(self, n, vc):
            s = ''.join(filter(None, vc)).strip()
            self.translation = s
            return s

        def visit_func(self, n, vc):
            self.kind = 'component'
            return functions[n.text.lower()]

        def visit_in_oper(self, n, vc):
            return in_ops[n.text.lower()]

        def visit_pre_oper(self, n, vc):
            return pre_ops[n.text.lower()]

        def visit_reference(self, n, vc):
            self.kind = 'component'
            id_str = vc[0]
            return id_str + '()'

        def visit_id(self, n, vc):
            return namespace[n.text]

        def visit_lookup_def(self, n, vc):
            """ This exists because vensim has multiple ways of doing lookups.
            Which is frustrating."""
            x_val = vc[4]
            pairs = vc[11]
            mixed_list = pairs.replace('(', '').replace(')', '').split(',')
            xs = mixed_list[::2]
            ys = mixed_list[1::2]
            string = "functions.lookup(%(x)s, [%(xs)s], [%(ys)s])" % {
                'x': x_val,
                'xs': ','.join(xs),
                'ys': ','.join(ys)
            }
            return string

        def visit_array(self, n, vc):
            if 'subs' in element and element['subs']:  # first test handles when subs is not defined
                coords = utils.make_coord_dict(element['subs'], subscript_dict, terse=False)
                dims = [utils.find_subscript_name(subscript_dict, sub) for sub in element['subs']]
                shape = [len(coords[dim]) for dim in dims]
                if ';' in n.text or ',' in n.text:
                    text = n.text.strip(';').replace(' ', '').replace(';', ',')
                    data = np.array([float(s) for s in text.split(',')]).reshape(shape)
                else:
                    data = np.tile(float(n.text), shape)
                datastr = np.array2string(data, separator=',').replace('\n', '').replace(' ', '')
                return textwrap.dedent("""\
                    xr.DataArray(data=%(datastr)s,
                                 coords=%(coords)s,
                                 dims=%(dims)s )""" % {
                    'datastr': datastr,
                    'coords': repr(coords),
                    'dims': repr(dims)})

            else:
                return n.text.replace(' ', '')

        def visit_subscript_list(self, n, vc):
            refs = vc[2]
            subs = [x.strip() for x in refs.split(',')]
            coordinates = utils.make_coord_dict(subs, subscript_dict)
            if len(coordinates):
                return '.loc[%s]' % repr(coordinates)
            else:
                return ' '

        def visit_build_call(self, n, vc):
            call = vc[0]
            args = vc[4]
            self.kind = 'component'
            arglist = [x.strip() for x in args.split(',')]
            name, structure = builders[call.strip().lower()](*arglist)
            self.new_structure += structure
            return name

        def visit__(self, n, vc):
            """ Handles whitespace characters"""
            return ''

        def generic_visit(self, n, vc):
            return ''.join(filter(None, vc)) or n.text

    parse_object = ExpressionParser(tree)

    return ({'py_expr': parse_object.translation,
             'kind': parse_object.kind,
             'arguments': ''},
            parse_object.new_structure)

In [43]:
tree

s = 'Total Costs '
Node('expr_type', s, 0, 12, children=[Node('expr', s, 0, 12, children=[RegexNode('_', s, 0, 0), Node('', s, 0, 0), RegexNode('_', s, 0, 0), Node('', s, 0, 12, children=[Node('reference', s, 0, 12, children=[RegexNode('id', s, 0, 11), RegexNode('_', s, 11, 12), Node('', s, 12, 12)])]), RegexNode('_', s, 12, 12), Node('', s, 12, 12)])])